In [17]:
import os
import tvm
from tvm.script import ir as I
from tvm.script import tir as T
from tvm import autotvm, auto_scheduler
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner
from tvm import meta_schedule as ms
from tvm.ir import IRModule
from tvm import relax
from tvm import rpc
from tvm.contrib import utils, ndk
x_shape = 4096
w_w_x = 512
w_s_x = 128
w_y = 11008
func_name = "main"
@I.ir_module
class ModuleSrc:
    @T.prim_func(private=False)
    def main(lv1611: T.Buffer((T.int64(512), T.int64(11008)), "uint32"),
             lv1612: T.Buffer((T.int64(128), T.int64(11008)), "float16"),
             lv1622: T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16"),
             p_output0_intermediate: T.Buffer((T.int64(1), T.int64(1), T.int64(11008)), "float16")):
        T.func_attr({"global_symbol": "main", "tir.noalias": T.bool(True)})
        # with T.block("root"):
        var_decode_intermediate = T.alloc_buffer((T.int64(4096), T.int64(11008)), "float16")
        var_matmul_intermediate = T.alloc_buffer(
            (T.int64(1), T.int64(1), T.int64(11008)), "float16"
        )
        compute = T.alloc_buffer((T.int64(1), T.int64(1), T.int64(11008)), "float16")
        for i, j in T.grid(T.int64(4096), T.int64(11008)):
            with T.block("decode"):
                v_i, v_j = T.axis.remap("SS", [i, j])
                T.reads(lv1611[v_i // T.int64(8), v_j], lv1612[v_i // T.int64(32), v_j])
                T.writes(var_decode_intermediate[v_i, v_j])
                var_decode_intermediate[v_i, v_j] = (
                    T.Cast(
                        "float16",
                        T.bitwise_and(
                            T.shift_right(
                                lv1611[v_i // T.int64(8), v_j],
                                T.Cast("uint32", v_i % T.int64(8)) * T.uint32(4),
                            ),
                            T.uint32(15),
                        ),
                    )
                    - T.float16(7)
                ) * lv1612[v_i // T.int64(32), v_j]
        for i0, i1, i2, k in T.grid(T.int64(1), T.int64(1), T.int64(11008), T.int64(4096)):
            with T.block("matmul"):
                v_i0, v_i1, v_i2, v_k = T.axis.remap("SSSR", [i0, i1, i2, k])
                T.reads(lv1622[v_i0, v_i1, v_k], var_decode_intermediate[v_k, v_i2])
                T.writes(var_matmul_intermediate[v_i0, v_i1, v_i2])
                with T.init():
                    var_matmul_intermediate[v_i0, v_i1, v_i2] = T.float16(0)
                var_matmul_intermediate[v_i0, v_i1, v_i2] = (
                    var_matmul_intermediate[v_i0, v_i1, v_i2]
                    + lv1622[v_i0, v_i1, v_k] * var_decode_intermediate[v_k, v_i2]
                )
        for i0, i1, i2 in T.grid(T.int64(1), T.int64(1), T.int64(11008)):
            with T.block("compute"):
                v_i0, v_i1, v_i2 = T.axis.remap("SSS", [i0, i1, i2])
                T.reads(var_matmul_intermediate[v_i0, v_i1, v_i2])
                T.writes(compute[v_i0, v_i1, v_i2])
                compute[v_i0, v_i1, v_i2] = T.sigmoid(
                    var_matmul_intermediate[v_i0, v_i1, v_i2]
                )
        for ax0, ax1, ax2 in T.grid(T.int64(1), T.int64(1), T.int64(11008)):
            with T.block("T_multiply"):
                v_ax0, v_ax1, v_ax2 = T.axis.remap("SSS", [ax0, ax1, ax2])
                T.reads(
                    var_matmul_intermediate[v_ax0, v_ax1, v_ax2],
                    compute[v_ax0, v_ax1, v_ax2],
                )
                T.writes(p_output0_intermediate[v_ax0, v_ax1, v_ax2])
                p_output0_intermediate[v_ax0, v_ax1, v_ax2] = (
                    var_matmul_intermediate[v_ax0, v_ax1, v_ax2]
                    * compute[v_ax0, v_ax1, v_ax2]
                )

@I.ir_module
class ModuleToManual:
    @T.prim_func(private=False)
    def main(lv1611: T.Buffer((T.int64(512), T.int64(11008)), "uint32"),
             lv1612: T.Buffer((T.int64(128), T.int64(11008)), "float16"),
             lv1622: T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16"),
             p_output0_intermediate: T.Buffer((T.int64(1), T.int64(1), T.int64(11008)), "float16")):
        T.func_attr({"global_symbol": "main", "tir.noalias": T.bool(True)})
        # with T.block("root"):
        var_decode_intermediate = T.alloc_buffer((T.int64(4096), T.int64(11008)), "float16")
        var_matmul_intermediate = T.alloc_buffer(
            (T.int64(1), T.int64(1), T.int64(11008)), "float16"
        )
        compute = T.alloc_buffer((T.int64(1), T.int64(1), T.int64(11008)), "float16")
        for i, j in T.grid(T.int64(4096), T.int64(11008)):
            with T.block("decode"):
                v_i, v_j = T.axis.remap("SS", [i, j])
                T.reads(lv1611[v_i // T.int64(8), v_j], lv1612[v_i // T.int64(32), v_j])
                T.writes(var_decode_intermediate[v_i, v_j])
                var_decode_intermediate[v_i, v_j] = (
                    T.Cast(
                        "float16",
                        T.bitwise_and(
                            T.shift_right(
                                lv1611[v_i // T.int64(8), v_j],
                                T.Cast("uint32", v_i % T.int64(8)) * T.uint32(4),
                            ),
                            T.uint32(15),
                        ),
                    )
                    - T.float16(7)
                ) * lv1612[v_i // T.int64(32), v_j]
        for i0, i1, i2, k in T.grid(T.int64(1), T.int64(1), T.int64(11008), T.int64(4096)):
            with T.block("matmul"):
                v_i0, v_i1, v_i2, v_k = T.axis.remap("SSSR", [i0, i1, i2, k])
                T.reads(lv1622[v_i0, v_i1, v_k], var_decode_intermediate[v_k, v_i2])
                T.writes(var_matmul_intermediate[v_i0, v_i1, v_i2])
                with T.init():
                    var_matmul_intermediate[v_i0, v_i1, v_i2] = T.float16(0)
                var_matmul_intermediate[v_i0, v_i1, v_i2] = (
                    var_matmul_intermediate[v_i0, v_i1, v_i2]
                    + lv1622[v_i0, v_i1, v_k] * var_decode_intermediate[v_k, v_i2]
                )
        for i0, i1, i2 in T.grid(T.int64(1), T.int64(1), T.int64(11008)):
            with T.block("compute"):
                v_i0, v_i1, v_i2 = T.axis.remap("SSS", [i0, i1, i2])
                T.reads(var_matmul_intermediate[v_i0, v_i1, v_i2])
                T.writes(compute[v_i0, v_i1, v_i2])
                compute[v_i0, v_i1, v_i2] = T.sigmoid(
                    var_matmul_intermediate[v_i0, v_i1, v_i2]
                )
        for ax0, ax1, ax2 in T.grid(T.int64(1), T.int64(1), T.int64(11008)):
            with T.block("T_multiply"):
                v_ax0, v_ax1, v_ax2 = T.axis.remap("SSS", [ax0, ax1, ax2])
                T.reads(
                    var_matmul_intermediate[v_ax0, v_ax1, v_ax2],
                    compute[v_ax0, v_ax1, v_ax2],
                )
                T.writes(p_output0_intermediate[v_ax0, v_ax1, v_ax2])
                p_output0_intermediate[v_ax0, v_ax1, v_ax2] = (
                    var_matmul_intermediate[v_ax0, v_ax1, v_ax2]
                    * compute[v_ax0, v_ax1, v_ax2]
                )

In [18]:
## ref to mlc-llm/dispatch/dispatch_tir_operator_adreno.py
def print_object_attr(cls):
    print(f"attr len = {len(cls.__dict__)}")
    for k, v in cls.__dict__.items():
        # if k.startswith("_") or k.startswith("__"):
        #     continue
        print(f"{k}: {v}")

def sch_fused_decode5_fused_matmul6_silu1(func):
    """
    Summarize
    --------------------------------
    gemv整体分析优化内容:
    1. 内存方面
        
        1.1 将参与多次计算的 `v` 搬运到shared_memory中
        
        1.2 将参与单次运算, 且较小的 quantized_weight_scale搬运到local memory中, 这一步感觉作用不大，可以做实验看下去掉的效果

        1.3 通过tiling, 使计算满足cache line, 提高cache命中
    
    2. 计算方面
        
        2.1 计算合并(内联), 如将dequantize操作与gemv合并, 将中间结果从local_memory -> register, 降低中间缓存及其内存搬运导致的耗时, 大幅度提高计算效率

        2.2 通过split和reorder操作实现tiling, 提高cache命中, 减少内存搬运

        2.3 绑定硬件线程(block & thread)

        2.4 使用vectorize, 利用SIMD指令集加速 load/store & ALU计算

        2.5 实际上还将内存搬运(global memory -> local/shared memory)与计算独立开来，提高内存搬运效率(需要看生成的源码如何实现)

        2.6 `v` 的shared memory搬移时, 使用cooperative_fetch, 暂时没搞明白如何实现, 需要生成的cuda代码

    3. 改进点?
        没有看到用unroll操作, 其余的好像都用到了 -- 用了 vectorize, 不需要unroll了
        
    opencl source
    ----------------------------------------------------------------
    // Function: main_kernel
    #ifdef cl_khr_fp16
    #pragma OPENCL EXTENSION cl_khr_fp16 : enable
    #elif defined(cl_amd_fp16)
    #pragma OPENCL EXTENSION cl_amd_fp16 : enable
    #else
    #error "Half precision floating point not supported by OpenCL implementation on your device." 
    #endif
    // lv1611 - w_int4
    // lv1612 - scale (group size = 32)
    // lv1622 - input x
    // kernel:  输入 [512, 11008], [128, 11008], [1,1,4096] -> [1,1 11008]
    //          工作组设置: blockIdx = 43, threadIdx = 64
    //          计算: output[4] = input[4096] x 4列W [1,1,4096] => [4]
    __kernel void main_kernel(__global uint* restrict lv1611, __global half* restrict lv1612, __global half* restrict lv1622, __global half* restrict p_output0_intermediate) {
    // 这里标注的是local内存, 即
    __local half lv1622_shared[4096];
    // 以下三个half4都是private memory, 如果resgiter足够的话, 会放到resgiter中, 如果不够会放到 local memory -> global memory
    half4 var_matmul_intermediate_local[1];
    half4 lv1612_local[1];
    uint4 lv1611_local[1];
    ## local_id范围[0,63], 64*8*8=4096, 即一个workgroup把 [1,1,4096]的输入都读到了shared_memory中
    for (int ax2_0 = 0; ax2_0 < 8; ++ax2_0) {
        vstore8(vload8(0, lv1622 + ((ax2_0 * 512) + ((convert_int(get_local_id(0))) * 8))), 0, lv1622_shared + ((ax2_0 * 512) + ((convert_int(get_local_id(0))) * 8)));
    }
    // 中间缓存初始化
    var_matmul_intermediate_local[0] = ((half4)((half)0.000000e+00f, (half)0.000000e+00f, (half)0.000000e+00f, (half)0.000000e+00f));
    barrier(CLK_LOCAL_MEM_FENCE);
    for (int k_0 = 0; k_0 < 128; ++k_0) {
        // load scale(group size = 32), shape[128, 11008]
        // 一个group_id处理256个数据, 一个local_id处理(k0 * 4)个数据(reudce) --> blockIdx = 43, threadIdx = 64, 即每次读取 [1, 4] 个Scale数据 -> [1, 4*32]个Weight的scale
        lv1612_local[0] = vload4(0, lv1612 + (((k_0 * 11008) + ((convert_int(get_group_id(0))) * 256)) + ((convert_int(get_local_id(0))) * 4)));
        for (int k_1 = 0; k_1 < 4; ++k_1) {
        // load w_int4, shape[512, 11008]
        // 一个group_id处理256个数据, 一个local_id处理(k1 * k0 * 4)个数据 --> blockIdx = 43, threadIdx = 64, 即每次读取 [1, 4] 个 Weight数据 -> [1, 4*8]个Wieght的量化数据
        lv1611_local[0] = vload4(0, lv1611 + ((((k_0 * 44032) + (k_1 * 11008)) + ((convert_int(get_group_id(0))) * 256)) + ((convert_int(get_local_id(0))) * 4)));
        // weigt: u32 X 4 -> 8 X int4 X 4
        for (int k_2 = 0; k_2 < 8; ++k_2) {
            // half4 ouput = output + (half4 lv1622_shared[k_0*32 + k_1*8 + k_2]) * half4
            var_matmul_intermediate_local[0] = (var_matmul_intermediate_local[0] + (((half4)(lv1622_shared[(((k_0 * 32) + (k_1 * 8)) + k_2)], lv1622_shared[(((k_0 * 32) + (k_1 * 8)) + k_2)], lv1622_shared[(((k_0 * 32) + (k_1 * 8)) + k_2)], lv1622_shared[(((k_0 * 32) + (k_1 * 8)) + k_2)])) * (((convert_half4(((lv1611_local[0]  >>  ((uint4)(((convert_uint(k_2)) * (uint)4), ((convert_uint(k_2)) * (uint)4), ((convert_uint(k_2)) * (uint)4), ((convert_uint(k_2)) * (uint)4))))  &  ((uint4)((uint)15, (uint)15, (uint)15, (uint)15))))) - ((half4)((half)7.000000e+00f, (half)7.000000e+00f, (half)7.000000e+00f, (half)7.000000e+00f))) * lv1612_local[0])));
        }
        }
    }
    vstore4((var_matmul_intermediate_local[0] * (((half4)((half)1.000000e+00f, (half)1.000000e+00f, (half)1.000000e+00f, (half)1.000000e+00f)) / (((half4)((half)1.000000e+00f, (half)1.000000e+00f, (half)1.000000e+00f, (half)1.000000e+00f)) + exp((((half4)((half)0.000000e+00f, (half)0.000000e+00f, (half)0.000000e+00f, (half)0.000000e+00f)) - var_matmul_intermediate_local[0]))))), 0, p_output0_intermediate + (((convert_int(get_group_id(0))) * 256) + ((convert_int(get_local_id(0))) * 4)));
    }
    """ 
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main") # decode([512,11008], [128, 11008]) -> [4096, 11008]
    b1 = sch.get_block(name="matmul", func_name="main") # matmul([4096], [4096, 11008], ) -> [11008]
    # matmul: 1, 1, 11008, 4096
    l2, l3, l4, l5 = sch.get_loops(block=b1)
    # l6 = 11008
    l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
    # v7 = 43, v8 = 64, v9 = 4
    v7, v8, v9 = sch.sample_perfect_tile(
        loop=l6, n=3, max_innermost_factor=4, decision=[43, 64, 4]
    )
    # print(sch.get(v7)) #可以打印值
    # spatial axes l6 = 11008 -> l10 = 43, l11 = 64, l12 = 4
    l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)

    # l5 = 4096 -> v13 = 128, v14 = 4, v15 = 8
    v13, v14, v15 = sch.sample_perfect_tile(
        loop=l5, n=3, max_innermost_factor=8, decision=[128, 4, 8]
    )
    # reduce l5 = 4096 -> l16 = 128, l17 = 4, l18 = 8
    l16, l17, l18 = sch.split(
        loop=l5, factors=[v13, v14, v15], preserve_unit_iters=True
    )
    # spatial loopRV: l10, l11, l12
    # reduce loopRV: l16, l17, l18
    ## 从这里来看规律：
    ###     spatial可以拆分，内层factor l12可以放到最内层循环
    ###     reduce可以做拆分，并移动到spatial外层循环和内层循环之间，效果上就是每个reduce position可以执行多次spatial操作(即spatial内层)
    ####        优化效果: 1. 在spatial内层操作时，输出的element不变，减少了输出内存的读取；但是对于W来说，需要隔行去读取element, 因此需要多计算几个reduce(列)以利用cache
    ####        优化效果: 2. 由于W shape = [4096, 11008], x[4096] * shape的操作，相当于x与每列W做运算，这会导致cache命中极低，导致cache thrashing(缓存抖动)
    ####            调整后, 对于W的读取，每一个k_inner(l17 * l18 = 4 * 8, fp16数据类型，正好是64bytes)块中会做s_inner(l12 = 4)次运算, 
    ####            ** 相当于4个cache line同时计算, 实际上4行这个数字取决于cache的大小，这里应该可以更大
    ####            ** 相当于，做了tiling操作, 为什么要将k_inner拆分，目前理解是可以用l18(8)来做vectorize的load/store/alu 指令(即SIMD)
    ####        优化效果: 3. 对于外层spatial可以绑定到硬件线程了
    sch.reorder(l10, l11, l16, l17, l18, l12)
    # l10 = 43, 绑定到blockIdx
    sch.bind(loop=l10, thread_axis="blockIdx.x")
    # l11 = 64, 绑定到threadIdx,  64对于adereno gpu来说有意义，因为cache line = 64bytes
    # l10*l11 = 2752, AD4X后的GPU最大的workgroup size(threadIdx.x * threadIdx.y * threadIdx.z) 一般为1024， 
    sch.bind(loop=l11, thread_axis="threadIdx.x")
    # 将decode过程内联到使用decode结果的block中去，这里就是将decode放到matmul中
    ## 从最终优化上来看：将 w = dequant(scale); o = x * w 合并为 => o = x * dequant(scale), 即将语句合并了
    ##      由于gemv中每个w值参与一次计算, 所以这一步可以节省中间缓存的内存搬运消耗，并可以节省memory, 理论上直接计算的中间结果会存在register，访存更快
    ##      @TODO 但是这个计算合并的操作是不是`compute_inline`做的，还不清楚 -- 看`compute_inline`的example是这样的
    ### 在compute_inline之前, block(b1).reads = [lv1622, var_decode_intermediate], writes = [var_matmul_intermediate]
    sch.compute_inline(block=b0)
    ## block(b1).reads = [lv1622, lv1611, lv1612], writes = [var_matmul_intermediate]
    ## 从reads的顺序来看, compute_inline会把b0.reads插到b1.reads后面,  同理writes应该也是这样, 但是这里没有writes, 所以没体现
    ### 也就是compute_inline把var_decode_intermediate中间变量给省去了，印证了之前的猜测
    ## b19指向的是 var_matmul_intermediate的赋值过程，即从global memory -> local memroy
    ## 也就是b19是一个block, 并且包含 loops, b19.loops(未经调整)=b19.shape
    b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    # reverse_compute_at 将一个消费者block(b19)移动到指定的loop(l11)中，并且重新生成该block包含的loops(暂时理解为绑定到该block的loops)
    ##   这样该block(b19)消费的buffer(write index 0, 即var_matmul_intermediate)区域就可以覆盖
    ##    指定loop(l11)下生产者blocks(一个或多个，这里指decode)生成的buffer区域(暂时理解为var_decode_intermediate)
    # b19: [1, 1, 11008]
    # b19插入后: [l10, l11, 1, 1, 1*1*11008/(l10*l11)] -> [43, 64, 1, 1, 4] 这个只是纯赋值的loop grid，在update时还要将reduce插入
    sch.reverse_compute_at(block=b19, loop=l11, preserve_unit_loops=True, index=-1)
    # 执行完reverse_compute_at, b1.reads和writes的内容没变
    # 将lv1611[512,11008]设置为local内存
    ## 这个内存设置奇怪, 只用一次, 不需要线放到local中
    b20 = sch.cache_read(block=b1, read_buffer_index=1, storage_scope="local")
    # 将lv1612[128,11008]设置为local内存
    ## 这个内存设置奇怪, 只用一次, 不需要线放到local中
    b21 = sch.cache_read(block=b1, read_buffer_index=2, storage_scope="local")
    # 将lv1622[1,1,4096]设置为local内存
    b22 = sch.cache_read(block=b1, read_buffer_index=0, storage_scope="shared")
    # 将一个生产者block(b22)移动到指定loop(l11)中，并且重新生成该block(b22)包含的loops, 这样该block(b22)生产的buffer区域就可以覆盖指定loop(l11)下消费者blocks消费的buffer区域
    ## preserve_unit_loops的作用: 如将(128, 129) compute_at (128)下，若为true则会有 for 128 (for(1, 129)), 办即保留shape中value为1的loop, 若为False, 则有for 128 (for(129))
    # b22: lv1622[1,1,4096]
    ## 猜测shape: [l10, l11, 1, 1, 4096/(l10 * l11)] @TODO 无法整除咋整.. 所以这里有个-1, 表示找最后一个可能的loop
    ## 观察到shape: [[l10, l11], [1,1,4096]] 即b22的初始化还在[l10,l11]的loop内，但是压根没按照这个计算, 这种情况得看下最终的cuda代码是如何处理的
    ##### for i0_i1_i2_fused_1 in T.thread_binding(T.int64(64), thread="threadIdx.x"):
    #####     lv1622_shared = T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16", scope="shared")
    #####     for ax0, ax1, ax2 in T.grid(T.int64(1), T.int64(1), T.int64(4096)):
    #####         with T.block("lv1622_shared"):
    #####             v0, v1, v2 = T.axis.remap("SSS", [ax0, ax1, ax2])
    #####             lv1622 = T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16")
    #####             T.reads(lv1622[v0, v1, v2])
    #####             T.writes(lv1622_shared[v0, v1, v2])
    #####             lv1622_shared[v0, v1, v2] = lv1622[v0, v1, v2]
    sch.compute_at(block=b22, loop=l11, preserve_unit_loops=True, index=-1)
    # print(sch.get(l11))
    v23 = sch.sample_categorical(
        candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=3
    )
    # 做标注，实际形式会在b22中生成 T.block_attr({"ann_key", "ann_value"})
    ## 标注了 lv1622[4096] cooperative_fetch:8
    sch.annotate(
        block_or_loop=b22, ann_key="meta_schedule.cooperative_fetch", ann_val=v23
    )
    # 这里有点奇怪，b20和b21是decode需要用到的buffer, 而l16 l17是reduce loop拆分的前两个subloop, 为什么会绑定到不同
    sch.compute_at(block=b20, loop=l17, preserve_unit_loops=True, index=-1)
    sch.compute_at(block=b21, loop=l16, preserve_unit_loops=True, index=-1)
    # @TODO 猜测b20_0: b20的原始loop: [512, 11008], 插入到l17后变为: [l10, l11, l16, l17, 512, 11008]
    # l10 = 43, l11 = 64, l16 = 128, l17 = 4 => [l10, l11, l16, l17, 1, 512*11008/(43*64*128*4)]
    ## 实际情况: 插入l17后变为 [43, 64, 128, 4, 1, 4]
    l24, l25, l26, l27, l28, l29 = sch.get_loops(block=b20)
    # print(sch.get(l28))#, sch.get(l25),sch.get(l26), sch.get(l27), sch.get(l28),sch.get(l29))
    # vectorize 4
    sch.vectorize(loop=l29)
    # b21: lv1612[128,11008]
    # b21 插入后: [l10, l11, l16, 1, 128*11008/(l10*l11*l16)] => [43, 64, 128, 1, 4 ]
    l30, l31, l32, l33, l34 = sch.get_loops(block=b21)
    sch.vectorize(loop=l34)
    # b19: [1, 1, 11008]
    # b19插入后: [l10, l11, 1, 1, 11008/(l10*l11)] -> [43, 64, 1, 1, 4]
    ## TODO暂时不知道为什么会多一个维度
    l35, l36, l37, l38, l39 = sch.get_loops(block=b19)
    # l39 = 4
    sch.vectorize(loop=l39)
    # l12 = 4
    sch.vectorize(loop=l12)
    # 包含reduction计算的即为reduction block
    # 将reduction block中的 Init block和update block拆分， 即将matmul中var_matmul_intermediate = float16(0)的操作拆分出来
    # 注意现在b1通过之前的compute_at包含了很多初始化的计算(@TODO 实际看只有b22的赋初始值的过程)
    # b40是b22的赋初始值过程 @TODO 没找到b20 b21 b22的内存转移操作在哪儿
    b40 = sch.decompose_reduction(block=b1, loop=l16)
    # b41只有计算sigmoid的过程
    b41 = sch.get_block(name="compute", func_name="main")
    # 将compute(sigmoid(z), z is the result of matmul block) inline消费者block中
    sch.compute_inline(block=b41)
    # b42包含 x*sigmoid
    b42 = sch.get_block(name="T_multiply", func_name="main")
    # 将T_multiply(x*y, where y is sigmoid(z), z is the result of matmul block) inline到它的生产者中
    ## 由于此时compute已经inline到T_multiply中了, 所以T_multiply = (x*sigmoid(z)), 整体移动到matmul中去
    sch.reverse_compute_inline(block=b42)
    # 标注scheudle的后处理开始，@TODO不知道作用 
    sch.enter_postproc()
    # 去掉 annotate, 从这里可以猜测`enter_postproc`应该对annotate信息做了处理
    sch.unannotate(block_or_loop=b22, ann_key="meta_schedule.cooperative_fetch")
    # 此时b22只包含 global memory -> shared memory的过程
    # 察到shape: [[l10, l11], [1,1,4096]] 即b22的初始化还在[l10,l11]的loop内
    l43, l44, l45, l46, l47 = sch.get_loops(block=b22)
    # print(sch.get(l43))
    # preserve_unit_iters作用不大，仅是否考虑计算 value为1 的loop
    l48, l49, l50 = sch.split(loop=l47, factors=[None, 64, 8], preserve_unit_iters=True)
    # l49 = 64, l50 = 8
    sch.vectorize(loop=l50)
    sch.bind(loop=l49, thread_axis="threadIdx.x")
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)


sch_manual = tvm.tir.Schedule(ModuleToManual)
# sch_fused_decode5_fused_matmul6_silu1(sch_manual.mod[func_name])
sch_manual.mod['main'] = sch_fused_decode5_fused_matmul6_silu1(sch_manual.mod[func_name])
# print(sch_manual.mod.script())
# rt_mod = tvm.build(sch_manual.mod, target="opencl")
# print(rt_mod.imported_modules[0].get_source())

In [19]:
### test examples
from tvm import tir
@T.prim_func
def before_compute_at(a: T.handle, c: T.handle) -> None:
    A = T.match_buffer(a, (128, 129), "float32")
    B = T.alloc_buffer((128, 129), "float32")
    C = T.match_buffer(c, (128, 129), "float32")
    for i, j in T.grid(128, 129):
        with T.block("B"):
            vi, vj = T.axis.remap("SS", [i, j])
            B[vi, vj] = A[vi, vj] * 2.0
    for i, j in T.grid(128, 129):
        with T.block("C"):
            vi, vj = T.axis.remap("SS", [i, j])
            C[vi, vj] = B[vi, vj] + 1.0


exam_sch = tir.Schedule(before_compute_at)
block = exam_sch.get_block("B")
loop, _ = exam_sch.get_loops(exam_sch.get_block("C"))
print(type(exam_sch.get(loop)))
exam_sch.compute_at(block, loop, preserve_unit_loops=True)
print(exam_sch.mod["main"].script())


<class 'tvm.tir.stmt.For'>
# from tvm.script import tir as T

@T.prim_func
def before_compute_at(A: T.Buffer((128, 129), "float32"), C: T.Buffer((128, 129), "float32")):
    # with T.block("root"):
    B = T.alloc_buffer((128, 129))
    for i in range(128):
        for ax0, ax1 in T.grid(1, 129):
            with T.block("B"):
                vi = T.axis.spatial(128, i + ax0)
                vj = T.axis.spatial(129, ax1)
                T.reads(A[vi, vj])
                T.writes(B[vi, vj])
                B[vi, vj] = A[vi, vj] * T.float32(2)
        for j in range(129):
            with T.block("C"):
                vi, vj = T.axis.remap("SS", [i, j])
                T.reads(B[vi, vj])
                T.writes(C[vi, vj])
                C[vi, vj] = B[vi, vj] + T.float32(1)


In [20]:
# exam 2
@T.prim_func
def before_split(a: T.handle, b: T.handle) -> None:
    A = T.match_buffer(a, (128, 128))
    B = T.match_buffer(b, (128, 128))
    for i, j in T.grid(128, 128):
        with T.block("B"):
            vi, vj = T.axis.remap("SS", [i, j])
            B[vi, vj] = A[vi, vj] * 2.0

split_sch = tir.Schedule(before_split)
i, j = split_sch.get_loops(split_sch.get_block("B"))
split_sch.split(i, factors=[2, 64,1], preserve_unit_iters =True)
print(split_sch.mod["main"].script())

# from tvm.script import tir as T

@T.prim_func
def before_split(A: T.Buffer((128, 128), "float32"), B: T.Buffer((128, 128), "float32")):
    # with T.block("root"):
    for i_0, i_1, i_2, j in T.grid(2, 64, 1, 128):
        with T.block("B"):
            vi = T.axis.spatial(128, i_0 * 64 + i_1 + i_2)
            vj = T.axis.spatial(128, j)
            T.reads(A[vi, vj])
            T.writes(B[vi, vj])
            B[vi, vj] = A[vi, vj] * T.float32(2)


In [21]:
# run and compare with cuda
import numpy as np
def _detect_local_cuda():
    dev = tvm.cuda()
    if not dev.exist:
        return None
    return tvm.target.Target(
        {
            "kind": "cuda",
            "max_shared_memory_per_block": dev.max_shared_memory_per_block,
            "max_threads_per_block": dev.max_threads_per_block,
            "thread_warp_size": dev.warp_size,
            "registers_per_block": 65536,
            "arch": "sm_" + tvm.cuda().compute_version.replace(".", ""),
        }
    )
# target = tvm.target.Target("cuda", host="llvm")
target = _detect_local_cuda()

print(target)
# 定义计算任务
dev = tvm.cuda(0)

num_flop = 1228406784
W_w_np = np.random.uniform(size=(w_w_x, w_y)).astype("uint32")
W_s_np = np.random.uniform(size=(w_s_x, w_y)).astype("float16")
Input_np = np.random.uniform(size=(1, 1, x_shape)).astype("float16")
Output_nd = tvm.nd.array(np.zeros((1, 1, w_y), dtype="float16"), dev)

cuda -keys=cuda,gpu -arch=sm_61 -max_num_threads=1024 -max_shared_memory_per_block=49152 -max_threads_per_block=1024 -registers_per_block=65536 -thread_warp_size=32


In [23]:

# cuda优化版本测试
manual_rt_mod = tvm.build(sch_manual.mod, target="cuda")
manual_evaluator = manual_rt_mod.time_evaluator("main", dev, number=100)
W_w_nd = tvm.nd.array(W_w_np, dev)
W_s_nd = tvm.nd.array(W_s_np, dev)
Input_nd = tvm.nd.array(Input_np, dev)
Output_nd = tvm.nd.array(np.zeros((1, 1, w_y), dtype="float16"), dev)
print("manual_evaluator GEMV-Blocking: %f GFLOPS" % (num_flop / manual_evaluator(W_w_nd, W_s_nd, Input_nd, Output_nd).mean / 1e9))
print(Output_nd.numpy())

manual_evaluator GEMV-Blocking: 197.832523 GFLOPS
[[[-0. -0. -0. ... -0. -0. -0.]]]


In [24]:
# cuda未优化版本测试
sch = tvm.tir.Schedule(ModuleSrc)
with target:
    src_gpu_mod = tvm.tir.transform.DefaultGPUSchedule()(sch.mod) ##
rt_mod = tvm.build(src_gpu_mod, target="cuda")
# print(rt_mod.get_source())
evaluator = rt_mod.time_evaluator("main", dev, number=100)
print("manual_evaluator GEMV-Blocking: %f GFLOPS" % (num_flop / manual_evaluator(W_w_nd, W_s_nd, Input_nd, Output_nd).mean / 1e9))
print(Output_nd.numpy())

manual_evaluator GEMV-Blocking: 196.482495 GFLOPS
[[[-0. -0. -0. ... -0. -0. -0.]]]


In [13]:
os.environ["TVM_NDK_CC"]="/home/sensetime/Android/Sdk/ndk/25.2.9519653/toolchains/llvm/prebuilt/linux-x86_64/bin/aarch64-linux-android33-clang++"
target = tvm.target.Target("opencl -device=adreno", host="llvm -mtriple=aarch64-linux-gnu")
device_key="android"
rpc_host = "10.4.236.32"
rpc_port = 9190
comp_target = tvm.target.Target("opencl", host="llvm -mtriple=aarch64-linux-android")  # TODO: Only support arm64 for now

def test_opencl(mod: tvm.IRModule, name_hint: str):
    # mod = tvm.lower(sch_manual.mod)
    print("Build ...")
    android_rt_mod = tvm.build(mod, target="opencl", target_host="llvm -mtriple=aarch64-linux-android")
    # print(android_rt_mod.imported_modules[0].get_source())
    temp = utils.tempdir()
    path_dso_cl = temp.relpath("dev_lib_cl.so")
    android_rt_mod.export_library(path_dso_cl, ndk.create_shared)

    print("Run GPU(OpenCL Flavor) test ...")
    # Establish remote connection with target hardware

    tracker = rpc.connect_tracker(rpc_host, rpc_port)
    remote = tracker.request(device_key, priority=0, session_timeout=60)
    print("Connect to device done.")
    dev = remote.cl(0)
    remote.upload(path_dso_cl)
    f1 = remote.load_module("dev_lib_cl.so")

    W_w_nd = tvm.nd.array(W_w_np, dev)
    W_s_nd = tvm.nd.array(W_s_np, dev)
    Input_nd = tvm.nd.array(Input_np, dev)
    Output_nd = tvm.nd.array(np.zeros((1, 1, w_y), dtype="float16"), dev)
    test_number=10
    time_f = f1.time_evaluator(f1.entry_name, dev, number=test_number)
    cost = time_f(W_w_nd, W_s_nd, Input_nd, Output_nd).mean
    print("evaluator[%s] GEMV-Blocking: %f ms with loop %d" % (name_hint, cost * 1000, test_number))
    print("evaluator[%s] GEMV-Blocking: %f GFLOPS" % (name_hint, num_flop / cost / 1e9))

    return Output_nd.numpy()

In [14]:
# 未优化版本opencl测试
from tvm import dlight as dl
sch = tvm.tir.Schedule(ModuleSrc)
with target:
    # src_gpu_mod = tvm.tir.transform.DefaultGPUSchedule()(sch.mod) ##
    mod_deploy = dl.ApplyDefaultSchedule(  # pylint: disable=not-callable
        dl.gpu.Matmul(),
        dl.gpu.GEMV(),
        dl.gpu.Reduction(),
        dl.gpu.GeneralReduction(),
        dl.gpu.Fallback(),
    )(sch.mod)
src_output = test_opencl(mod_deploy, "source")
print(src_output)


Build ...
Run GPU(OpenCL Flavor) test ...
Connect to device done.
evaluator[source] GEMV-Blocking: 2.538496 ms with loop 10
evaluator[source] GEMV-Blocking: 483.911255 GFLOPS
[[[-0. -0. -0. ... -0. -0. -0.]]]


In [15]:
#优化版本opencl测试
opt_output = test_opencl(sch_manual.mod, "opted")
print(opt_output)
np.testing.assert_equal(opt_output, src_output)

Build ...
Run GPU(OpenCL Flavor) test ...
Connect to device done.
evaluator[opted] GEMV-Blocking: 0.516531 ms with loop 10
evaluator[opted] GEMV-Blocking: 2378.185062 GFLOPS
[[[-0. -0. -0. ... -0. -0. -0.]]]


In [ ]:
import numpy as np
target = tvm.target.Target("opencl -device=adreno", host="llvm -mtriple=aarch64-linux-gnu")
device_key="android"
rpc_host = "10.158.176.30"
rpc_port = 5001
# remote = autotvm.measure.request_remote(device_key, "10.158.176.30", 5001, timeout=10000)
# dev = remote.device(str(target), 0)

# num_flop = 1228406784
# W_np = np.random.uniform(size=(512, vocab_size)).astype("uint32")
# S_np = np.random.uniform(size=(128, vocab_size)).astype("float16")
# Input_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# # Output_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# W_nd = tvm.nd.array(W_np, dev)
# S_nd = tvm.nd.array(S_np, dev)
# Input_nd = tvm.nd.array(Input_np, dev)
# Output_nd = tvm.nd.array(np.zeros((1, 1, vocab_size), dtype="float32"), dev)

In [ ]:
rpc_config = ms.runner.RPCConfig(tracker_host=rpc_host, tracker_port=rpc_port, tracker_key = device_key)
runner= ms.runner.RPCRunner(rpc_config)
# ms.builder.LocalBuilder()
sch = tvm.tir.Schedule(ModuleSrc)
database = ms.tune_tir(
    mod=ModuleSrc,
    target=target,
    max_trials_global=64,
    num_trials_per_iter=64,
    work_dir="./tune_first",
    cost_model="xgb",
    runner = runner
)
print(len(database))
sch1 = ms.tir_integration.compile_tir(database, sch.mod, target)
print(type(sch1))

In [ ]:
from tvm.script import relax as R
@I.ir_module
class Module:
    @R.function
    def main(A: R.Tensor((3, 4), dtype="float32"), B: R.Tensor((4, 5), dtype="float32")):
        with R.dataflow():
            lv: R.Tensor((3, 5), dtype="float32") = R.matmul(A, B)
            gv: R.Tensor((3, 5), dtype="float32") = lv
            R.output(gv)
        return gv

In [ ]:
## auto_scheduler test
from tvm import auto_scheduler
import numpy as np
a_np = np.random.rand(3, 4).astype("float32")
b_np = np.random.rand(4, 5).astype("float32")
a_nd = tvm.runtime.NDArray(a_np)
b_nd = tvm.runtime.NDArray(b_np)
sch = tvm.tir.Schedule(Module)

params = {"A": a_np, "B": b_np}
## 报错，这里只支持relay
# tasks = auto_scheduler.extract_tasks(sch.mod, params, target=target)
tasks = ms.relax_integration.extract_tasks(sch.mod, target=target, params=params)
print(len(tasks))

In [ ]:

from mod_deploy import Module as ModuleAll
params_all = {}
tasks_all = auto_scheduler.extract_tasks(ModuleAll, params_all, target=target)
print(len(tasks_all))

In [ ]:
import numpy as np
log_file = "tune.json"
def _detect_local_cuda():
    dev = tvm.cuda()
    if not dev.exist:
        return None
    return tvm.target.Target(
        {
            "kind": "cuda",
            "max_shared_memory_per_block": dev.max_shared_memory_per_block,
            "max_threads_per_block": dev.max_threads_per_block,
            "thread_warp_size": dev.warp_size,
            "registers_per_block": 65536,
            "arch": "sm_" + tvm.cuda().compute_version.replace(".", ""),
        }
    )
# target = tvm.target.Target("cuda", host="llvm")
target = _detect_local_cuda()

print(target)
# 定义计算任务
dev = tvm.cuda(0)

num_flop = 1228406784
W_np = np.random.uniform(size=(512, vocab_size)).astype("uint32")
S_np = np.random.uniform(size=(128, vocab_size)).astype("float16")
Input_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# Output_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
W_nd = tvm.nd.array(W_np, dev)
S_nd = tvm.nd.array(S_np, dev)
Input_nd = tvm.nd.array(Input_np, dev)
Output_nd = tvm.nd.array(np.zeros((1, 1, vocab_size), dtype="float32"), dev)
sch = tvm.tir.Schedule(ModuleSrc)
new_mod = sch.mod


In [ ]:
# task = auto_scheduler.SearchTask(func=sch.mod['fused_fused_decode11_fused_matmul5_cast2'], args=sch.mod['fused_fused_decode11_fused_matmul5_cast2'].params, target=target)

# tune_option = auto_scheduler.TuningOptions(
#     num_measure_trials=10,
#     measure_callbacks=[auto_scheduler.RecordToFile(log_file)],
#     verbose=2,
# )


database = ms.tune_tir(
    mod=new_mod,
    target=target,
    max_trials_global=64,
    num_trials_per_iter=64,
    work_dir="./tune_45593_1",
    cost_model="xgb"
)
print(len(database))
sch1 = ms.tir_integration.compile_tir(database, new_mod, target)
print(type(sch1))

In [ ]:
# print(sch1.trace)
# print(sch1.mod.script())
rt_mod = tvm.build(sch1.mod, target="cuda")

evaluator = rt_mod.time_evaluator("main", dev, number=100)

print("evaluator GEMV-Blocking: %f GFLOPS" % (1228406784 / evaluator(W_nd, S_nd, Input_nd, Output_nd).mean / 1e9))




In [ ]:

record_database = ms.Database.create(kind='json', work_dir='./tune_45593_1')


In [ ]:
record_sch = ms.tir_integration.compile_tir(record_database, new_mod, target)

record_rt_mod = tvm.build(record_sch.mod, target="cuda")

record_evaluator = record_rt_mod.time_evaluator("main", dev, number=20)

print("evaluator GEMV-Blocking: %f GFLOPS" % (num_flop / record_evaluator(W_nd, S_nd, Input_nd, Output_nd).mean / 1e9))
print(record_sch.trace)
print(record_sch.mod.script())

In [ ]:
from typing import TYPE_CHECKING, Dict, List, Optional, Union, Callable
from tvm import runtime
if TYPE_CHECKING:
    import numpy as np  # type: ignore
    from tvm.ir import IRModule
    from tvm.meta_schedule.runner import EvaluatorConfig, RPCConfig
    from tvm.runtime import Device, Module, NDArray
    from tvm.target import Target
    from tvm.runtime.vm import Executable


def f_measurement(
    rt_mod: runtime.Module, device: runtime.ndarray.Device, input_data: Dict[str, runtime.NDArray]
):
    vm = relax.VirtualMachine(rt_mod, device=device)
    vm.save_function("main", "measure_func", **input_data, include_return=False)
    evaluator = vm.time_evaluator(
        func_name="measure_func",
        dev=device,
        repeat=100,
        number=1,
        min_repeat_ms=500,
    )
    return evaluator()

def run_module_via_rpc(
    rpc_config: "RPCConfig",
    lib: Union["Module", "Executable"],
    dev_type: str,
    args: Union[Dict[int, "np.ndarray"], Dict[str, "np.ndarray"]],
    continuation: Callable,
    backend: Optional[str] = "graph",
):
    """Execute a tvm.runtime.Module on RPC remote"""
    # pylint: disable=import-outside-toplevel
    import os
    import tempfile

    from tvm.contrib.tar import tar
    from tvm.runtime import ndarray

    # pylint: enable=import-outside-toplevel

    with tempfile.TemporaryDirectory() as tmp_dir:
        # filename = os.path.join(tmp_dir, "tvm_tmp_mod." + tar.output_format)
        filename = os.path.join(tmp_dir, "tvm_tmp_mod." + "so")
        if backend == "vm":
            code, lib = lib.save(filename, fmt="so")
        from tvm.contrib import ndk
        lib.export_library(filename, ndk.create_shared)
        session = rpc_config.connect_server()
        print(type(session._sess))
        session.upload(filename)
        _, filename = os.path.split(filename)
        rt_mod = session.load_module(filename)
        
        if backend == "vm":
            rt_mod = session.get_function("runtime.Load_Executable")(code, rt_mod)
            # rt_mod = session.get_function("runtime.module.loadfile_relax.Executable")(filename)
        dev = session.device(dev_type=dev_type, dev_id=0)
        # print(dev)
        # create the remote runtime module
        print(rt_mod)
        print(rt_mod['main'])
        from tvm.contrib import graph_executor as runtime
        module = runtime.GraphModule(rt_mod["main"](dev))
        print(module)
        for k, v in args.items():
            module.set_input(k, tvm.nd.array(v))
        return module.run()
        # nd_args = {k: ndarray.array(v, dev) for k, v in args.items()}
        nd_args = {k: ndarray.empty(v.shape, v.dtype, dev) for k, v in args.items()}
        return continuation(rt_mod, dev, nd_args)